In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import yfinance as yf

In [ ]:
# Fetch data for a specific stock, e.g., Apple Inc. with the ticker symbol 'AAPL'
data = yf.download('AAPL', start='2020-01-01', end='2024-02-05')

# Display the first few rows of the dataframe
print(data.head())

ticker = yf.Ticker('AAPL')

# Get historical market data
hist = ticker.history(period="max")

# Show actions (dividends, splits)
actions = ticker.actions

# Show dividends
dividends = ticker.dividends

# Show financials
financials = ticker.financials

data = yf.download('AAPL', start='2020-01-01', end='2024-02-05', interval="1mo")

[*********************100%%**********************]  1 of 1 completed


                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2020-01-02  74.059998  75.150002  73.797501  75.087502  73.152649  135480400
2020-01-03  74.287498  75.144997  74.125000  74.357498  72.441444  146322800
2020-01-06  73.447502  74.989998  73.187500  74.949997  73.018684  118387200
2020-01-07  74.959999  75.224998  74.370003  74.597504  72.675278  108872000
2020-01-08  74.290001  76.110001  74.290001  75.797501  73.844353  132079200


[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Use only the 'Close' column
close_prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
close_prices_normalized = scaler.fit_transform(close_prices)

# Convert to PyTorch tensors
close_prices_normalized = torch.FloatTensor(close_prices_normalized).view(-1)


In [ ]:

def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Define the sequence length
seq_length = 5
train_data = create_inout_sequences(close_prices_normalized, seq_length)
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)


In [ ]:
class StockPredictor(nn.Module):
    def __init__(self, input_size, hidden_layer_size=100, output_size=1):
        super(StockPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_layer_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = StockPredictor(seq_length)


In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for i in range(epochs):
    for seq, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(seq)
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%10 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')


epoch:   0 loss: 0.01224436
epoch:  10 loss: 0.04678084
epoch:  20 loss: 0.08376815
epoch:  30 loss: 0.00006177
epoch:  40 loss: 0.00001832
epoch:  50 loss: 0.00056583
epoch:  60 loss: 0.03044098
epoch:  70 loss: 0.09287038
epoch:  80 loss: 0.00219140
epoch:  90 loss: 0.00450917


In [ ]:
with torch.no_grad():
    pred_seq = close_prices_normalized[-seq_length:]
    pred_seq = pred_seq.view(1, -1)
    predicted_price_normalized = model(pred_seq)
    predicted_price = scaler.inverse_transform(predicted_price_normalized.numpy())
    print(f'Predicted price: {predicted_price}')


Predicted price: [[182.2541]]
